# Tableau and player form

Here we create a csv file that is used for making further Tableau visualizations. 

In [1]:
# recent gameweek
gameweek = 36

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch player stats data
all_df = pd.read_csv('../data/fpl/data_week1.csv', index_col=0)
all_df['gameweek'] = 1
all_df['xPoints'] = all_df['adjusted points']

for week in range(2,gameweek+1):
    filepath = '../data/fpl/data_week' + str(week) + '.csv'
    df = pd.read_csv(filepath, index_col=0)
    df['gameweek'] = week
    column_name = 'xPoints week ' + str(week)
    df['xPoints'] = df[column_name]
    all_df = all_df.append(df,sort='False')

all_df.shape

(22915, 342)

In [4]:
# calculate player form
for i in [5,10,15,20]:
    gameweeks = np.arange(gameweek-i+1,gameweek+1)
    column_name = 'form ' + str(i)
    all_df.loc[all_df['gameweek']==gameweek, column_name] = (all_df[all_df['gameweek'].isin(gameweeks)].groupby(['id'])['xPoints'].sum() / (all_df.loc[(all_df['gameweek']==gameweek),'games played'].round() - all_df.loc[(all_df['gameweek']==gameweek-i),'games played'].round()) )
    # dirty fix in case there are inf results
    all_df.loc[(all_df['gameweek']==gameweek) & np.isinf(all_df[column_name]), column_name] = np.nan

In [5]:
# top form players
all_df[all_df['gameweek']==gameweek][['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10', 'form 15', 'form 20']].sort_values(by=['form 10'], ascending=False).head(20)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10,form 15,form 20
id,,,,,,,,,,
352,Jack Robinson,8.888889,24,2.7,26.764235,3.010976,8.460717,8.460717,8.460717,8.460717
388,Kane,32.857143,230,7.0,211.646278,6.441408,6.900000,7.146656,6.444279,6.527090
259,Alexander-Arnold,33.863636,149,4.4,167.107123,4.934707,7.041294,6.938435,5.647404,5.462626
254,Salah,35.238095,222,6.3,212.900775,6.041779,6.805299,6.550783,5.828222,5.577951
233,Iheanacho,23.181818,102,4.4,90.568658,3.906883,5.554099,6.496722,5.666961,5.044700
91,Wood,31.162791,134,4.3,135.462987,4.346947,5.478582,6.434522,5.646427,4.719418
272,De Bruyne,24.150943,128,5.3,152.259524,6.304496,NaN,6.433506,4.940441,5.754247
500,Havertz,25.714286,90,3.5,92.815201,3.609480,6.794151,6.182698,5.534861,4.524039
303,Martial,22.058824,75,3.4,75.420303,3.419054,NaN,6.009953,3.821534,3.427970


In [6]:
# save data
filepath = '../data/fpl/all_data.csv'
all_df.to_csv(filepath)

filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
all_df[all_df['gameweek']==gameweek].to_csv(filepath)